Note:

- notebook di ingestion che si trasformerà in uno script (dag) della pipeline
- prenderà i dati dall'API e farà l'ingestion in un data lake (al momento istnaza di EC2)
  - vedere la parte di data lake. L'idea al momento è di effettuare dei trigger ad evennto o schedulate dell'API che va in append con i nuovi record in file parques storicizzati in un data lake
  - in futuro sarà in data lake che potrà storicizzare altre tipologie di dati
- i dati verranno letti dallo step successivo per la fase di preprocessamento per le quali si avrà un primo layer a DB dei dati


In [14]:
import os
root_path = os.path.abspath(".")

if root_path.startswith("/content"):

  from google.colab import drive
  drive.mount('/content/drive')

  os.chdir('/content/drive/MyDrive/GitHub_Repositories/ML_InvestmentManagement')

  root_path = os.path.abspath(".")

  !pip install  boto3 awswrangler awscli plotly fundamentalanalysis --quiet

/content/drive/MyDrive/GitHub_Repositories/ML_InvestmentManagement
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
path_config_aws = os.path.abspath("config/awscli.ini")

if not os.path.exists(path_config_aws):
    text = """
  [ml_invest]
  aws_access_key_id = <...>
  aws_secret_access_key =  <...>
  region = eu-west-3
  """
    with open(path_config_aws, "w") as f:
        f.write(text)
else:
    print("awscli.ini already exist!")

awscli.ini already exist!


In [16]:
import os
!export AWS_SHARED_CREDENTIALS_FILE=path_config_aws

os.environ['AWS_SHARED_CREDENTIALS_FILE'] = path_config_aws
print(os.environ['AWS_SHARED_CREDENTIALS_FILE'])

/content/drive/MyDrive/GitHub_Repositories/ML_InvestmentManagement/config/awscli.ini


In [13]:
import configparser

config = configparser.ConfigParser()
config.read("./config/credential.ini")

fmp_key = config.get("API_KEYS", "FMP")

In [14]:
from openbb_terminal.sdk import openbb

# help(openbb.keys.fmp)
openbb.keys.fmp(key=fmp_key, persist=True)
openbb.keys.mykeys(show=False)

,Key
API,
ETHPLORER_KEY,*******
KEY_FINANCIALMODELINGPREP,*******


In [17]:
import pandas as pd
import numpy as np
import awswrangler as wr
import boto3
import os
import yaml
import fundamentalanalysis as fa

pd.set_option("display.max_columns", None)

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

from tqdm import tqdm
from configparser import ConfigParser
from configparser import ExtendedInterpolation

from utils.financial_modelling_prep.details import available_companies
from utils.financial_modelling_prep.financial_statements import (
    get_and_save_financial_statament_data_fmp,
    get_index_hist_price,
    get_hist_price,
)

from datetime import date

In [18]:
from urllib.request import urlopen
from urllib.error import HTTPError
import json
import certifi


def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

In [19]:
with open(os.path.abspath("config/1_data_ingestion.yaml")) as file:
    config_ingestion = yaml.safe_load(file)

In [20]:
boto3.setup_default_session(profile_name="ml_invest")

NameError: name 'ticker' is not defined

In [21]:
raw_s3_bucket = "ml-invest-bucket"
raw_path_dir = "financial_modeling_prep"
raw_path = f"s3://{raw_s3_bucket}/{raw_path_dir}/"

In [22]:
today = date.today().strftime("%Y-%m-%d")

In [23]:
database = config_ingestion["database"]
tables = config_ingestion["tables"]

save_file = config_ingestion["save_file"]
download = config_ingestion["download"]
limit = config_ingestion["limit"]

Forecasting Toolkit is disabled. To use the Forecasting features please install the toolkit following the instructions here: https://docs.openbb.co/sdk/quickstart/installation/

Portfolio Optimization Toolkit is disabled. To use the Optimization features please install the toolkit following the instructions here: https://docs.openbb.co/sdk/quickstart/installation/

,Key
API,
ETHPLORER_KEY,*******
KEY_FINANCIALMODELINGPREP,*******


In [24]:
# TODO: create utils methods to handle AWS
client = boto3.client("glue")

# List the databases in the Glue Data Catalog
response = client.get_databases()

database = response["DatabaseList"][0]["Name"]

response_table = client.get_tables(DatabaseName=database)

# TODO print more information about tables
print(f"Database: {database}\n")
print("Tables:")
df_tables = wr.catalog.tables()
df_tables

Database: financial_data

Tables:


,Database,Table,Description,TableType,Columns,Partitions
0,financial_data,2_abt_base,,EXTERNAL_TABLE,"date, ticker, adjclose, volume, unadjustedvolu...",
1,financial_data,2_balance_sheet_statement_process,,EXTERNAL_TABLE,"cik, fillingdate, calendaryear, period, cashan...",
2,financial_data,2_cash_flow_statement_process,,EXTERNAL_TABLE,"cik, fillingdate, calendaryear, period, netcas...",
3,financial_data,2_income_statement_process,,EXTERNAL_TABLE,"cik, fillingdate, calendaryear, period, revenu...",
4,financial_data,3_1_fe_targets,,EXTERNAL_TABLE,"date, ticker, adjclose, cik, diff_return_mean_...",
5,financial_data,alerts_score,,EXTERNAL_TABLE,"ticker, fillingdate, calendaryear, period, ale...",
6,financial_data,balance_sheet_statement,,EXTERNAL_TABLE,"index, reportedcurrency, cik, fillingdate, acc...",
7,financial_data,cash_flow_statement,,EXTERNAL_TABLE,"index, reportedcurrency, cik, fillingdate, acc...",
8,financial_data,daily_index_price,,EXTERNAL_TABLE,"ticker, date, adjclose, unadjustedvolume, vwap",
9,financial_data,daily_price,,EXTERNAL_TABLE,"date, open, high, low, close, adjclose, volume...",


### Read data from API


In [26]:
# list of companies
companies = available_companies(api_key)
companies.reset_index(inplace=True)

In [27]:
companies_nasdaq = companies[
    (companies["exchangeShortName"] == "NASDAQ") & (companies["type"] == "stock")
]
list_ticker = companies_nasdaq.symbol.to_list()

#### Download Historical Marketcap


In [ ]:
start_date = "2000-01-01"

In [ ]:
df_marketcap = pd.DataFrame()
for ticker in tqdm(list_ticker):
    try:
        url = f"https://financialmodelingprep.com/api/v3/historical-market-capitalization/{ticker}?&apikey={api_key}"
        data_mod = get_jsonparsed_data(url)

        data_mod = pd.DataFrame(data_mod)
        data_mod["date"] = pd.to_datetime(data_mod["date"])

        data_mod = data_mod[data_mod["date"] > start_date]
        df_marketcap = pd.concat([df_marketcap, data_mod], ignore_index=True)

    except Exception as e:
        pass
    # df_marketcap.to_csv(f"data/financial_model_prep/hist_marketcap_{today}.csv", index=False)

df_marketcap.rename(columns={"symbol": "ticker"}, inplace=True)
df_marketcap.to_csv(
    f"data/financial_model_prep/hist_marketcap_{today}.csv", index=False
)
table = "marketcap"
wr.s3.to_parquet(
    df=df_marketcap,
    path=raw_path + table,
    dataset=True,
    mode="overwrite",
    database=database,
    table=table,
)

### Economic Indicator


Inflation Rate:

- https://fred.stlouisfed.org/series/T10YIE


In [28]:
quarterly_economic_indicators_list = [
    "GDP",
    "realGDP",
    "nominalPotentialGDP",
    "realGDPPerCapita",
    "commercialBankInterestRateOnCreditCardPlansAllAccounts",
]
monthly_economic_indicators_list = [
    "federalFunds",
    "CPI",
    "retailSales",
    "consumerSentiment",
    "durableGoods",
    "unemploymentRate",
    "totalNonfarmPayroll",
    "industrialProductionTotalIndex",
    "newPrivatelyOwnedHousingUnitsStartedTotalUnits",
    "totalVehicleSales",
    "retailMoneyFunds",
    "smoothedUSRecessionProbabilities",
    "3MonthOr90DayRatesAndYieldsCertificatesOfDeposit",
]
weekly_economic_indicators_list = [
    "initialClaims",
    "30YearFixedRateMortgageAverage",
    "15YearFixedRateMortgageAverage",
]
daily_economic_indicators_list = ["inflationRate"]

In [29]:
list_economic_indicator = daily_economic_indicators_list
list_economic_indicator.extend(weekly_economic_indicators_list)
list_economic_indicator.extend(monthly_economic_indicators_list)
list_economic_indicator.extend(quarterly_economic_indicators_list)

In [30]:
df_economic_indicator = pd.DataFrame()

for economic_indicator in list_economic_indicator:
    try:
        url = f"https://financialmodelingprep.com/api/v4/economic?name={economic_indicator}&from=1999-11-01&to=2023-02-01&apikey={api_key}"
        data_mod = get_jsonparsed_data(url)

        data_mod = pd.DataFrame(data_mod)

        print(f"{economic_indicator} - {data_mod.shape}")
        data_mod["date"] = pd.to_datetime(data_mod["date"])

        data_mod.rename(columns={"value": economic_indicator}, inplace=True)
        data_mod.set_index("date", inplace=True)

        df_economic_indicator = pd.concat([df_economic_indicator, data_mod], axis=1)

    except Exception as e:
        print(e)
        pass

inflationRate - (5026, 2)
initialClaims - (1213, 2)
30YearFixedRateMortgageAverage - (1214, 2)
15YearFixedRateMortgageAverage - (1215, 2)
federalFunds - (279, 2)
CPI - (278, 2)
retailSales - (278, 2)
consumerSentiment - (278, 2)
durableGoods - (278, 2)
unemploymentRate - (279, 2)
totalNonfarmPayroll - (279, 2)
industrialProductionTotalIndex - (278, 2)
newPrivatelyOwnedHousingUnitsStartedTotalUnits - (278, 2)
totalVehicleSales - (279, 2)
retailMoneyFunds - (278, 2)
smoothedUSRecessionProbabilities - (278, 2)
3MonthOr90DayRatesAndYieldsCertificatesOfDeposit - (277, 2)
GDP - (92, 2)
realGDP - (92, 2)
nominalPotentialGDP - (93, 2)
realGDPPerCapita - (92, 2)
commercialBankInterestRateOnCreditCardPlansAllAccounts - (93, 2)


In [32]:
df_economic_indicator = df_economic_indicator.fillna(method="ffill")

In [33]:
df_economic_indicator = df_economic_indicator.dropna()

In [41]:
df_economic_indicator.reset_index(inplace=True)

In [44]:
wr.s3.to_parquet(
    df=df_economic_indicator,
    path=raw_path + "1_economic_indicators",
    dataset=True,
    mode="overwrite",
    database=database,
    table="1_economic_indicators",
)

{'paths': ['s3://ml-invest-bucket/financial_modeling_prep/1_economic_indicators/bdb8faf0bc864a66958cbf04b71d2946.snappy.parquet'],
 'partitions_values': {}}

#### Download Profile


In [ ]:
# PROFILE


df_profile_tot = pd.DataFrame()

for ticker in tqdm(list_ticker):
    profile = fa.profile(ticker, api_key).T

    df_profile_tot = pd.concat([df_profile_tot, profile], ignore_index=True)


wr.s3.to_parquet(
    df=df_profile_tot,
    path=raw_path + "profile",
    dataset=True,
    mode="overwrite",
    database=database,
    table="1_profile",
)

In [ ]:
for table in tables:
    print(table)

    if limit != 0:
        df_old = wr.s3.read_parquet_table(database=database, table=table)
        print(f"N. records table {table}: {df_old .shape}")

        unique_ticker = list(df_old["ticker"].unique())

        new_ticker = [
            ticker for ticker in tqdm(list_ticker) if ticker not in unique_ticker
        ]

        print(f"New ticker are {len(new_ticker)}")
    else:
        new_ticker = []

    df_new_statement = get_and_save_financial_statament_data_fmp(
        download, table, list_ticker, new_ticker, save_file=save_file, limit=limit
    )

    print(f"Data updated: {df_new_statement.shape}")

    # df_new_statement.columns = [col.lower() for col in df_new_statement.columns]
    # df_new_statement = df_new_statement[df_old.columns]

    if limit != 0:
        df_new_statement = pd.concat([df_old, df_new_statement], ignore_index=False)

        print(f"Concat: {df_new_statement.shape}")

        # converto in int altrimenti non fa correttamete il sort
        df_new_statement["calendaryear"] = df_new_statement["calendaryear"].astype(
            np.int64
        )

        df_new_statement = df_new_statement.drop_duplicates(
            ["ticker", "calendaryear", "period"], keep="last"
        )

        print(f"drop_duplicates: {df_new_statement.shape}")

        df_new_statement = df_new_statement.sort_values(
            ["ticker", "calendaryear", "period"]
        ).reset_index(drop=True)

        for col, type in dict(df_old.dtypes).items():
            df_new_statement[col] = df_new_statement[col].astype(type)

    wr.s3.to_parquet(
        df=df_new_statement,
        path=raw_path + table,
        dataset=True,
        mode="overwrite",
        database=database,
        table=table,
    )

100%|██████████| 5987/5987 [1:20:24<00:00,  1.24it/s]


Shape df cash_flow_statement: (201654, 40)
Data updated: (201654, 40)


100%|██████████| 5987/5987 [1:27:15<00:00,  1.14it/s]


Shape df balance_sheet_statement: (200808, 54)
Data updated: (200808, 54)


In [ ]:
wr.s3.to_parquet(
    df=df_new_statement,
    path=raw_path + table,
    dataset=True,
    mode="overwrite",
    database=database,
    table=table,
)

{'paths': ['s3://ml-invest-bucket/financial_modeling_prep/income_statement/c55b5509a6f34d309834380d05384a43.snappy.parquet'],
 'partitions_values': {}}

In [ ]:
df_new_statement = df_new_statement[df_new_statement["accepteddate"] >= "2000-01-01"]

In [ ]:
today = date.today().strftime("%Y-%m-%d")
table = "daily_price"

df_old = wr.s3.read_parquet_table(database=database, table=table)
print(f"N. records table {table}: {df_old .shape}")

unique_ticker = list(df_old["ticker"].unique())


new_ticker = [ticker for ticker in tqdm(list_ticker) if ticker not in unique_ticker]

N. records table daily_price: (10607642, 5)


NameError: name 'list_ticker' is not defined

In [ ]:
df_index_price = get_index_hist_price(True, "2000-01-01", today)
df_price = get_hist_price(True, list_ticker, [], "2000-01-01", today)

Shape df index hist_price: (5801, 14)


100%|██████████| 5987/5987 [4:04:00<00:00,  2.45s/it]  


Shape df hist_price: (13118647, 14)


In [ ]:
df_index_price = get_index_hist_price(True, "2000-01-01", today)

Shape df index hist_price: (5805, 14)


In [ ]:
df_price = pd.read_csv("data/financial_model_prep/hist_price_20230124.csv")

In [ ]:
df_price["date"] = pd.to_datetime(df_price["date"])

In [ ]:
df_price_fin = df_price[df_price["date"] >= "2000-01-01"]

In [ ]:
df_price_fin.rename(columns={"symbol": "ticker"}, inplace=True)
# df_index_price.rename(columns = {"symbol":"ticker"}, inplace=True)

In [ ]:
# df_price = df_price[['ticker','date', 'adjClose', ,'unadjustedVolume','changePercent', 'vwap']]
# df_index_price = df_index_price[['ticker','date', 'adjClose', 'unadjustedVolume', 'vwap']]

In [ ]:
wr.s3.to_parquet(
    df=df_price_fin,
    path=raw_path + "daily_price",
    dataset=True,
    mode="overwrite",
    database=database,
    table="daily_price",
)

{'paths': ['s3://ml-invest-bucket/financial_modeling_prep/daily_price/31f62a43bcaf469ca1b5c9cb4916525d.snappy.parquet'],
 'partitions_values': {}}

In [ ]:
wr.s3.to_parquet(
    df=df_index_price,
    path=raw_path + "daily_index_price",
    dataset=True,
    mode="overwrite",
    database=database,
    table="daily_index_price",
)

{'paths': ['s3://ml-invest-bucket/financial_modeling_prep/daily_index_price/8001250962ec4b2faa86760ede7abcbc.snappy.parquet'],
 'partitions_values': {}}